In [12]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from sklearn.metrics import accuracy_score, r2_score
from models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:

df = pd.read_csv(datafile)
df["date"] = pd.to_datetime(df["day_survey"])
df['C'] = df['date'].apply(lambda date: COVIDStatus.POST_COVID if date > date_covid else COVIDStatus.PRE_COVID)

df_head = df.head(5).copy()

df.rename(columns=reverse_ema_dictionary, inplace=True)
df.set_index(["uid", "date"], inplace=True)
df.dropna(subset=ema + physical + social + sleep, inplace=True)

sets_df = pd.read_parquet(sets_file, engine='pyarrow')

In [14]:
index = 51
model_rows = [sets_df.iloc[index]]
#for index, model_row in sets_df.iterrows():

alphas = [0.2, 0.05, 0.4]
gammas = [0.5, 0.2]

for alpha in alphas:
    for gamma in gammas:
        #for model_row in model_rows:
        for index, model_row in sets_df.iterrows():

            print(f"Modeling yreatment:{full_dictionary[model_row['treatment']]} on outcome:{full_dictionary[model_row['outcome']]}")
            # people can't change their demographics.. except maybe OS :)
            if model_row['treatment'].startswith('D'):
                print(f"Skippnig demographic treatment {model_row['treatment']}")
                continue

            if model_row['outcome'] not in ["Y2", "Y3"]:
                print("Only doing PHQ right now")
                continue
            wbm = WBKernelModel(
                data=df,
                gamma=gamma,
                alpha=alpha,
                #activation='relu',
                #max_iter=1000,
                #hidden_layer_sizes=(32,8,4),
                treatment=model_row['treatment'],
                outcome=model_row['outcome'],
                separating_set=model_row['sets'].tolist(),
                name=f"row:{index}"
                )

            print(f"gamma={gamma}, alpha={alpha}, pre_rsq={wbm.pre_r_squared} post_r_sq={wbm.post_r_squared}")

Modeling yreatment:race on outcome:pam
Skippnig demographic treatment D2
Modeling yreatment:cohort year on outcome:pam
Skippnig demographic treatment D4
Modeling yreatment:studying on outcome:pam
Only doing PHQ right now
Modeling yreatment:in house on outcome:pam
Only doing PHQ right now
Modeling yreatment:sports on outcome:pam
Only doing PHQ right now
Modeling yreatment:traveling on outcome:pam
Only doing PHQ right now
Modeling yreatment:time in social location on outcome:pam
Only doing PHQ right now
Modeling yreatment:visits on outcome:pam
Only doing PHQ right now
Modeling yreatment:duration unlocked phone in social locations on outcome:pam
Only doing PHQ right now
Modeling yreatment:gender on outcome:phq2_score
Skippnig demographic treatment D1
Modeling yreatment:excercise on outcome:phq2_score
Binarizing using median value (excercise median=1.10e+04)
Directory 20240527_215523_010565 created.
gamma=0.5, alpha=0.2, pre_rsq=(0.6536288639530945, 0.23953194565527935) post_r_sq=(0.699829

KeyboardInterrupt: 

In [ ]:
results_df = aggregate_results_from_subfolders('.')
results_df.to_csv('results.csv')